In [ ]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community \
  scikit-image


In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

In [ ]:
prompt = """

Answer the user's query based on the context below.
If you don't know the answer based on the context
provided, please respond, I don't know.

---
Context: {context}
---
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
        ("system", prompt),
        ("user", "{query}")
])

In [ ]:
prompt_template.input_variables

In [ ]:
prompt_template.messages

In [ ]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

prompt_template_2 = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(prompt),
        HumanMessagePromptTemplate.from_template("{query}")
])

In [ ]:
prompt_template_2.messages

In [ ]:
prompt_template_2.input_variables

In [ ]:
from langchain.chat_models import init_chat_model


llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature=0.0) 

In [ ]:
pipeline = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template_2 
    | llm)

In [ ]:
context = """
Aurelio AI is an AI company developing tooling for AI
engineers. There focus is on the language AI with the team having strong
expertise in building AI agents and a strong background in information
retreival.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI 
Platform providing engineers with tooling to help them build with 
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI become langchain expert in September 2024 after a long 
track record of delivering AI solutions built with Langchain ecosystem.

"""

query ="What does Aurelio AI do?"

In [ ]:
pipeline.invoke({"query": query, "context": context})

In [ ]:
pipeline.invoke({"query": "What does Microsoft do", "context": context})

## Few Short Prompting

In [ ]:
example_prompt = ChatPromptTemplate.from_messages([ 
        HumanMessagePromptTemplate.from_template("{input}"),
        AIMessagePromptTemplate.from_template("{output}")
])

In [ ]:
examples = [
    {"input": "Here is the query #1", "output": "Here is the answer #1"},
    {"input": "Here is the query #2", "output": "Here is the answer #2"},
    {"input": "Here is the query #3", "output": "Here is the answer #3"}    
]

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

print(few_shot_prompt.format())

In [ ]:
new_system_prompt = """

Answer the user's query based on the context below.
If you don't know the answer based on the context
provided, please respond, I don't know.

Always answser in markdown format. When doing so please
provide headers, short summaries, follow with bullet 
points, then conclude.

---
Context: {context}
---
"""

prompt_template_markdown = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(new_system_prompt),
        HumanMessagePromptTemplate.from_template("{query}")
])
 

pipeline_markdown = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | prompt_template_markdown 
    | llm)

out = pipeline_markdown.invoke({"query": query, "context": context}).content

print(out)

In [ ]:
from IPython.display import display, Markdown

display(Markdown(out))

In [ ]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [ ]:
few_shot_prompt_new = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples
)

out = few_shot_prompt_new.format()

display(Markdown(out))

In [ ]:
few_shot_prompt_template = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(new_system_prompt),
        FewShotChatMessagePromptTemplate(
            example_prompt = example_prompt,
            examples = examples
        ),
        HumanMessagePromptTemplate.from_template("{query}")
])

In [ ]:
few_shot_prompt_template

In [ ]:
pipeline_few_shot = (
    {
        "query": lambda x: x["query"],
        "context": lambda x: x["context"]
    }
    | few_shot_prompt_template 
    | llm)

out = pipeline_few_shot.invoke({"query": query, "context": context}).content

display(Markdown(out))

## Chain of Thought

In [ ]:
no_cot_system_prompt = """
Be a helpful assistant and answer the user's question.

You MUST answer the question directly without any other
text or explanation.

"""

no_cot_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(no_cot_system_prompt),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [ ]:
query = "How many keystrokes are needed to type the numbers from 1 to 500?"

no_cot_pipeline = (
    { "query": lambda x: x["query"]}
    | no_cot_prompt_template
    | llm
)

no_cot_result = no_cot_pipeline.invoke({"query": query}).content
print(no_cot_result)

In [ ]:
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question you must:

- List systematically and precisely detail all
  subproblems that needs to be solved to answer the
  question.
- Solve each subproblem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to 
  provide the final answer. 

"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(cot_system_prompt),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [ ]:
cot_pipeline = (
    { "query": lambda x: x["query"]}
    | cot_prompt_template
    | llm
)

cot_result = cot_pipeline.invoke({"query": query}).content
display(Markdown(cot_result))

In [ ]:
default_system_prompt = """
Be a helpful assistant and answer the user's question.
"""

no_cot_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(default_system_prompt),
    HumanMessagePromptTemplate.from_template("{query}")
])

In [ ]:
default_pipeline = (
    { "query": lambda x: x["query"]}
    | cot_prompt_template
    | llm
)

default_result = default_pipeline.invoke({"query": query}).content
display(Markdown(default_result))